In [24]:
#
# cs590-Module7-mongo.ipynb - This is a Jupyter Notebook designed to 
# connect to a MongoDB database and provide CRUD operations.
#
# This notebook uses a supplemental configuration file that has the
# following format:
# 
# [mongodb]
# host=localhost
# database=delisweet
# collection=REPLACE_WITH_YOUR_COLLECTION_AS_NEEDED
# user=''
# password=REPLACE_WITH_YOUR_PASSWORD_FOR_THE_DATABASE_USER
# port=27017
#
# We also build a config function to read the data from the
# configuration file to make the code easier to maintain and more
# secure. 
#
# Remember: You should _NEVER_ embed a password in your 
# source code!
#
#------------------------------------------------------------------
# Change History
#------------------------------------------------------------------
# Version   |   Description
#------------------------------------------------------------------
#    1          Initial Development
#------------------------------------------------------------------

import pymongo
from pymongo import MongoClient
from bson.objectid import ObjectId
from configparser import ConfigParser

#
# db - variable to identify the database we are communicating with
#
db = 'delisweet'

#
# col - variable identify the collection we are working with
#
col = ''

#
# config - This function is designed to read the data from our
# configuration file to allow us to setup the connection to our
# MongoDB database.
#
# Please Note: We create this method genericly so that it can
# be reused with other configuration data separated by section
# as key-value pairs
#
# Parameters:
#
#   filename: The name of the file that contains the configuration data
#   section:  The name of the section within the configuration file
#             for which we are retrieving data
# 
# Returns:
#   A dictionary with the key-value pairs from the configuration file.
#
def config(filename, section):
    # Initialize a configuration parser
    cp = ConfigParser()
    
    # Acquire data from the configuration file
    cp.read(filename)
    
    # Import configuration section as a dictionary
    block = {}
    
    # Check to make sure that the section exists in our config file
    if cp.has_section(section):
        # Read the entire section
        items = cp.items(section)
        #print(items)
        
        # Loop through config items to create the dictionary
        for item in items:
            block[item[0]] = item[1]
    else:
        # If we have gotten here the section did not exist in the
        # config file, so we raise an exception
        raise Exception(f"Section {section} was not found in the {filename} file!")
    
    # Return the dictionary to the calling program
    return block


#
# displayConfig - This function is designed to test the config function.
# This function will call the config function with a filename and a section
# obtain the resulting data, and then print the retrieved information to
# the screen. This can be useful for debugging and testing purposes.
#
# Parameters:
#
#   filename: The name of the file that contains the configuration data
#   section:  The name of the section within the configuration file
#             for which we are retrieving data
# 
def displayConfig(filename, section):
    # Obtain configuration data from the config file
    confData = config(filename, section)
    
    # Display results
    print(confData)
    

#
# dbConnect - function designed to connect to the database. This function
# leverages the config method to pull the necessary data from our database
# configuration file.
#
# Parameters:
#
#   filename: The name of the file that contains the configuration data
#   section:  The name of the section within the configuration file
#             for which we are retrieving data
# 
# Returns a Connection to the databse or null in the event of a problem
#
def dbConnect(filename, section):
    # Obtain configuration data from the config file
    confData = config(filename, section)

    # create connection variable
    conn = None

    # Create a try/except block for the connection. This is important
    # in order to keep from blowing up the application in the event that
    # we cannot connect to the database.
    try:     
        # Setup the connection to the MongoDB server
        conn = MongoClient('mongodb://%s:%s@%s:%d' % (confData["user"],confData["password"],confData["host"],int(confData["port"])))
        database = conn['%s' % confData["database"]]
        collection = database['%s' % confData["collection"]]

        # Return the database connection for use in the application
        # Don't forget to close the connection when done.
        return conn
    except(Exception, pymongo.errors.ConnectionFailure) as error:
        # We had a problem, so lets see what it was
        print(error)
        return None
    
#
# dbTest - a utility method to test connection to the database
# This method will connect to the database with the information from
# the configuration file, validate that the connection was made to the 
# Appropriate database, print the result, and then close the database
# connection. This function is useful for debugging initial connections. 
# 
# Parameters:
#
#   filename: The name of the file that contains the configuration data
#   section:  The name of the section within the configuration file
#             for which we are retrieving data
#
def dbTest(filename, section):
    # create our connection
    print('Connecting to our MongoDB database...')
    conn = dbConnect(filename, section)
    database = conn[db]
    collection = database[col]
    

    # Create a try/except block for the connection. This is important
    # in order to keep from blowing up the application in the event that
    # we cannot connect to the database.
    try:     
        # How many documents are in our collection?
        numDocs = collection.count_documents({})
        print(f'There are {numDocs} in the {col} collection')
    except(Exception, pymongo.errors.ConnectionFailure) as error:
        # We had a problem, so lets see what it was
        print(error)
    finally:
        # Everything worked well, so we need to close the connection
        if conn is not None:
            conn.close()


#
# create - CRUD Method used to create a new record in a database. 
# This method will be used to create a new record to be added to a database.
# Because MongoDB is a document database, this will add a record to a
# specific collection.
#
# Parameters:
#     data: This should always be a list of dictionaries. This allows
#           the create method to insert one or more records into the
#           collection
#     conn: The database connection to use create the record
#
def createRecord(data, conn):
    # TODO: Create the code necessary to perform a create operation
    # against your MongoDB database.
    #
    # Your code must use the function signature listed above and it
    # must provide the correct test output when this notebook is run.
    raise NotImplementedError

#
# testCreateRecord - Method used to test the createRecord function.
# This function will exercise the createRecord function by attempting to add
# a Nutrient Description record to the collection. This function will
# only work with the Nutrient database, it is not portable.
#
# Parameters:
#   filename: The name of the file that contains the configuration data
#   section:  The name of the section within the configuration file
#             for which we are retrieving data
#
def testCreateRecord(filename, section):
    # create our connection
    print('Connecting to our MongoDB database...')
    conn = dbConnect(filename, section)

    # Setup the Dictionary

    data = [ 
            {   "row": {
                "Nutrient code": "777",
                "Nutrient description": "Zinc",
                "Nutrient description abbrev": "Zn",
                "Nutrient unit": "mcg",
                "Date added": "05/08/2025",
                "Last modified": "05/08/2025"
                }
            }
        ]

    # Add record to the collection
    createRecord(data,conn)
    print('Record Added')

    # Cleanup and close connection
    conn.close()


#
# readRecord - CRUD Method used to read a record from a database. 
# This method will be used to read a new record from a collection.
# Because MongoDB is a document database, this operation is specific
# to a document collection
#
# Parameters:
#     clause: partial dictionary containing the information to find
#     conn: The database connection to use create the record
#    
def readRecord(clause, conn):
    # TODO: Create the code necessary to perform a read operation
    # against your MongoDB database.
    #
    # Your code must use the function signature listed above and it
    # must provide the correct test output when this notebook is run.
    raise NotImplementedError


#
# testReadRecord - Method used to test the readRecord function.
# This function will exercise the readRecord function by attempting to read
# a Nutrient Description record from the collection. This function will
# only work with the Nutrient database, it is not portable.
#
# Parameters:
#   filename: The name of the file that contains the configuration data
#   section:  The name of the section within the configuration file
#             for which we are retrieving data
#
def testReadRecord(filename, section):
    # create our connection
    print('Connecting to our MongoDB database...')
    conn = dbConnect(filename, section)
    
    # Setup dictionary to find data
    clause = { "row.Nutrient code":  "777"}

    # Find records if they exist
    records = readRecord(clause, conn)
    
    # Display results
    print('Records Retrieved:')
    print(records)

    # Cleanup and close connection
    conn.close()

#
# updateRecord - CRUD Method used to update a record from a database. 
# This method will be used to update a record from a collection.
# Because MongoDB is a document database, this will update a operate on
# a record(s) in a collection.
#
# Parameters:
#     select: The dictionary representing the selection criteria for records
#             to be updated
#     clause: The dictionary indicating the update to be made to the records
#             selected by the filter
#     conn: The database connection to use create the record
#    
def updateRecord(select, clause, conn):
    # TODO: Create the code necessary to perform an update operation
    # against your MongoDB database.
    #
    # Your code must use the function signature listed above and it
    # must provide the correct test output when this notebook is run.
    raise NotImplementedError

#
# testUpdateRecord - Method used to test the updateRecord function.
# This function will exercise the updateRecord function by attempting to update
# a Nutrient Description record. This function will
# only work with the Nutrient database, it is not portable.
#
# Parameters:
#   filename: The name of the file that contains the configuration data
#   section:  The name of the section within the configuration file
#             for which we are retrieving data
#
def testUpdateRecord(filename, section):
    # create our connection
    print('Connecting to our MongoDB database...')
    conn = dbConnect(filename, section)

    # Setup filter to find records
    select = { "row.Nutrient code":  "777"}

    # Setup dictionary to perform updates
    clause = { "$set": { "row.Nutrient description": "ZINC" } }
    
    # Process test
    res = updateRecord(select, clause, conn)   

    # Print results
    print(f'{res} record(s) have been updated.')
    conn.close()


#
# deleteRecord - CRUD Method used to delete a record from a database. 
# This method will be used to delete a record from a database collection.
# Because MongoDB is a document database, this will remove a record from a
# collection.
#
# NOTE: USE WITH CAUTION! THIS METHOD WILL DESTROY DATA
#
# Parameters:
#     select: The dictionary used to select the record(s) to delete
#     conn: The database connection to use create the record
#    
def deleteRecord(select, conn):
    # TODO: Create the code necessary to perform a delete operation
    # against your MongoDB database.
    #
    # Your code must use the function signature listed above and it
    # must provide the correct test output when this notebook is run.
    raise NotImplementedError

#
# testDeleteRecord - Method used to test the deleteRecord function.
# This function will exercise the deleteRecord function by attempting to remove
# a record from the collection. This function will
# only work with the Nutrient database, it is not portable.
#
# Parameters:
#   filename: The name of the file that contains the configuration data
#   section:  The name of the section within the configuration file
#             for which we are retrieving data
#
def testDeleteRecord(filename, section):
    # create our connection
    print('Connecting to our PostgreSQL database...')
    conn = dbConnect(filename, section)

    # setup the dictionary to select the record(s) to remove
    select = { "row.Nutrient code":  "777"}
    
    # execute record removal
    ret = deleteRecord(select, conn)    

    # Report results
    print(f'{ret} record(s) were removed from the collection.')
    conn.close()

if __name__ == '__main__':
    displayConfig('database.conf','mongodb')
    dbTest('database.conf','mongodb')
    testCreateRecord('database.conf','mongodb')
    testReadRecord('database.conf','mongodb')
    testUpdateRecord('database.conf','mongodb')
    testDeleteRecord('database.conf','mongodb')

{'host': 'localhost', 'database': 'Nutrition', 'collection': 'nutrition', 'user': 'cs590', 'password': 'cs590password', 'port': '27017'}
Connecting to our MongoDB database...
There are 194755 in the nutrition collection
Connecting to our MongoDB database...
Record Added
Connecting to our MongoDB database...
Records Retrieved:
[{'_id': ObjectId('6825edf4d4eb44f29673dab4'), 'row': {'Nutrient code': '777', 'Nutrient description': 'Zinc', 'Nutrient description abbrev': 'Zn', 'Nutrient unit': 'mcg', 'Date added': '05/08/2025', 'Last modified': '05/08/2025'}}, {'_id': ObjectId('6825edfdd4eb44f29673dab7'), 'row': {'Nutrient code': '777', 'Nutrient description': 'Zinc', 'Nutrient description abbrev': 'Zn', 'Nutrient unit': 'mcg', 'Date added': '05/08/2025', 'Last modified': '05/08/2025'}}, {'_id': ObjectId('6825f136d4eb44f29673daba'), 'row': {'Nutrient code': '777', 'Nutrient description': 'Zinc', 'Nutrient description abbrev': 'Zn', 'Nutrient unit': 'mcg', 'Date added': '05/08/2025', 'Last mo